<a href="https://colab.research.google.com/github/akravc/nnsp20/blob/master/labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [0]:
# FUNCTION TAKES A FILENAME (HAS TO BE MOUNTED ON YOUR GOOGLE DRIVE)
# RETURNS A TENSOR OF ROWS IN THE FILE
# n is the number of lines in file

def rows (filename, n):
  trace = open(filename, 'r')
  lines = trace.readlines() 

  data = np.empty([n,4])
  for i in range(n):
    line = lines[i]
    data[i] = np.array(list(map(int, line.split())))

  return torch.tensor(data, dtype=torch.int32)

In [0]:
tensor100 = rows('/content/drive/My Drive/100k.txt', 100000)
tensor600 = rows('/content/drive/My Drive/600k.txt', 600000)

In [0]:
#TAKES A 10-TIMESTEP CHUNK OF DATA
#STARTING AT TIMESTEP TSTART
#N IS ROWS IN THE FILE

#THIS IS VERY CLEARLY HORRIBLY INEFFICIENT
#CAN IMPROVE ON THIS LATER
#OR NOT

def chunk(filetensor, tstart, n):
  # 100k file cuts off mid t=35, so round down to 
  # closest multiple of 10 + 1
  # similar with 600k
  maxtime = 0
  if n == 100:
    maxtime = 31
  if n == 600:
    maxtime = 211
  
  if (tstart + 10) > maxtime:
    print("10 timesteps ahead is too far!")
  else:
    output = []
    for row in filetensor:
      time = row[0]
      if time >= tstart and time <= (tstart + 10):
        output.append(np.array(row))
    return torch.tensor(np.array(output))

In [0]:
zeroten = chunk(tensor100, 0, 100)

In [0]:
#OUTPUTS DICTIONARY SUCH THAT
#FOR EACH OBJECT ID IN TSTART
#LABELED 1 IF IT APPEARS IN THE NEXT 10 STEPS
#LABELED 0 IF IT DOES NOT APPEAR
def label(chunk):
  tstart = chunk[0][0]
  tend = tstart + 10

  # begin dictionary with 
  # all objects accessed in tstart
  # assume we dont' see it again
  output = {}
  for row in chunk:
    if row[0] == tstart:
      obj_id = row[1]
      output[int(obj_id)] = 0

  #go thru all accesses in the next 10 timesteps
  #put the ones that appear with a 1, and remove them
  #from object list
  for row in chunk:
    if row[0] > tstart:
      obj_id = int(row[1])
      if obj_id in output.keys():
        output[obj_id] = 1

  return output

In [0]:
#INPUT FILE NAME
#OUTPUT FILE NAME
#N is lines in file

def generate_labels(inputfile, outputfile, n):
  maxtime = 0
  if n == 100:
    maxtime = 21
  if n == 600:
    maxtime = 201

  tensor = rows(inputfile, n*1000)

  # GET all labels for each timestep
  # store in dictionary of dictonaries
  # { 0 : { 234: 1}, .. }
  # where 0 is timestep, 234 is obj id, 1 is label
  dic = {}
  for i in range(maxtime):
    ch = chunk(tensor, i, n)
    dic[i] = label(ch)
  
  #open file to write and write to it
  output = open(outputfile, 'w')
  for row in tensor:
    time = int(row[0])
    if time >= maxtime:
      break
    obj_id = int(row[1])
    obj_size = int(row[2])
    extra = int(row[3])
    lab = dic[time][obj_id]
    line = [str(time), " ", str(obj_id), " ", str(obj_size), " ", str(extra), " ", str(lab), "\n"]
    output.writelines("".join(line))

  output.close()

In [20]:
generate_labels('/content/drive/My Drive/100k.txt', "/content/drive/My Drive/100labeled.txt", 100)

all labeled


In [0]:
generate_labels('/content/drive/My Drive/600k.txt', "/content/drive/My Drive/600labeled.txt", 600)